# MLflow + whylogs Integration

In this notebook, we will explore the [MLflow](https://mlflow.org/) integration in `whylogs`.

This example uses the data from [MLflow's tutorial](https://mlflow.org/docs/latest/tutorials-and-examples/tutorial.html) for demonstration purposes.

This tutorial showcases how you can use the whylogs integration to:
* Capture data quality metrics while training a linear regression model in `mlflow`
* Extract whylogs data back into an in-memory format from the MLflow backend
* Visualize this data

# Getting Started
To run this tutorial:
* Install [conda](https://conda.io/projects/conda/en/latest/user-guide/install/index.html)
* Create a new environment with conda via `conda create --name whylogs-mlflow python=3.8`
    * You'll need to activate the environment with `conda activate whylogs-mlflow`
    * You'll need to install pip into the Conda environment `conda install pip`
    * To make the environment work with Jupyter notebooks, run `pip install ipykernel` to install the kernel module
    * Install the environment as a Jupyter notebook kernel via `python -m ipykernel install --user --name=whylogs-mlflow`
* Install MLflow with scikit-learn via `pip install mlflow[extras]`
* Install whylogs with matplotlib via `pip install whylogs[viz]`
* You can also install the necessary libraries separately:
    * MLflow: `pip install mlflow`
    * whylogs: `pip install whylogs`
    * scikit-learn: `pip install scikit-learn`
    * matplotlib: `pip install matplotlib`
* In your notebook, ensure you select `whylogs-mlflow` as your kernel

# Setup
First, we want to filter out noisy warnings

In [1]:
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')


In [2]:
import random
import time

import pandas as pd
import mlflow
import whylogs
from whylogs import get_or_create_session

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

# Enable whylogs Integration

Enable whylogs in MLflow to allow storing whylogs statistical profiles with every run. This method returns `True` if whylogs is able to patch MLflow.

In [3]:
assert whylogs.__version__ >= "0.1.13" # we need 0.1.13 or later for MLflow integration
session = get_or_create_session(".whylogs_mlflow.yaml")
whylogs.enable_mlflow(session)

True

# Dataset Preparation

Download and prepare the UCI wine quality dataset. We sample the test dataset further to represent batches of data produced every second.

In [4]:
data_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
data = pd.read_csv(data_url, sep=";")
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [5]:
# Split the data into training and test sets
train, test = train_test_split(data)

We can quickly get a sense of the shape of the training dataset by profiling it with whylogs. Later, we can compare the baseline data metrics to the profiles of the batches as they flow through our model.

If you'd like to learn more about whylogs, check out our [introductory notebook](https://github.com/whylabs/whylogs-examples/blob/mainline/python/GettingStarted.ipynb).

In [6]:
from whylogs import get_or_create_session

with get_or_create_session() as session:
    summary = session.profile_dataframe(train, "training-data").flat_summary()['summary']

summary

WARN: Missing config


,column,count,null_count,bool_count,numeric_count,max,mean,min,stddev,nunique_numbers,...,stddev_token_length,quantile_0.0000,quantile_0.0100,quantile_0.0500,quantile_0.2500,quantile_0.5000,quantile_0.7500,quantile_0.9500,quantile_0.9900,quantile_1.0000
0,free sulfur dioxide,1199.0,0.0,0.0,1199.0,68.00000,15.648457,1.00000,10.127920,56.0,...,0.0,1.00000,3.0000,4.0000,7.00000,14.0000,21.0000,35.0000,48.0000,68.00000
1,alcohol,1199.0,0.0,0.0,1199.0,14.00000,10.441118,8.40000,1.056323,60.0,...,0.0,8.40000,9.0000,9.2000,9.50000,10.2000,11.1000,12.5000,13.3000,14.00000
2,density,1199.0,0.0,0.0,1199.0,1.00369,0.996731,0.99007,0.001885,392.0,...,0.0,0.99007,0.9922,0.9937,0.99568,0.9967,0.9978,0.9998,1.0022,1.00369
3,chlorides,1199.0,0.0,0.0,1199.0,0.61000,0.086515,0.01200,0.044167,132.0,...,0.0,0.01200,0.0420,0.0530,0.07000,0.0790,0.0900,0.1230,0.3870,0.61000
4,citric acid,1199.0,0.0,0.0,1199.0,1.00000,0.269183,0.00000,0.196366,78.0,...,0.0,0.00000,0.0000,0.0000,0.09000,0.2600,0.4300,0.6000,0.7100,1.00000
5,total sulfur dioxide,1199.0,0.0,0.0,1199.0,289.00000,45.745204,6.00000,32.470115,135.0,...,0.0,6.00000,8.0000,11.0000,22.00000,37.0000,60.0000,110.0000,148.0000,289.00000
6,residual sugar,1199.0,0.0,0.0,1199.0,15.50000,2.577064,0.90000,1.465876,85.0,...,0.0,0.90000,1.4000,1.6000,1.90000,2.2000,2.6000,5.5000,8.8000,15.50000
7,volatile acidity,1199.0,0.0,0.0,1199.0,1.33000,0.529108,0.12000,0.177753,135.0,...,0.0,0.12000,0.1900,0.2700,0.39500,0.5200,0.6400,0.8450,1.0200,1.33000
8,pH,1199.0,0.0,0.0,1199.0,4.01000,3.315038,2.74000,0.156388,86.0,...,0.0,2.74000,2.9200,3.0700,3.21000,3.3100,3.4000,3.5800,3.7100,4.01000
9,quality,1199.0,0.0,0.0,1199.0,8.00000,5.636364,3.00000,0.826044,6.0,...,0.0,3.00000,4.0000,5.0000,5.00000,6.0000,6.0000,7.0000,8.0000,8.00000


Now that we've taken a peek at our training data metrics, there's one last item on our to-do list: split the test data into batches, so we can feed them through our model later on.

In [7]:
# Relocate predicted variable "quality" to y vectors
train_x = train.drop(["quality"], axis=1).reset_index(drop=True)
test_x = test.drop(["quality"], axis=1).reset_index(drop=True)
train_y = train[["quality"]].reset_index(drop=True)
test_y = test[["quality"]].reset_index(drop=True)

subset_test_x = []
subset_test_y = []
num_batches = 20
for i in range(num_batches):
    indices = random.sample(range(len(test)), 5)
    subset_test_x.append(test_x.loc[indices, :])
    subset_test_y.append(test_y.loc[indices, :])

# Train the model
We'll train an ElasticNet model using scikit-learn, then run this model for each of the batches of data, logging the model parameters, mean absolute error, and whylogs data metrics.

Note that whylogs profiler data is automatically logged when `mlflow.end_run()` is called (implicitly or explicitly).

In [8]:
# Create an MLflow experiment for our demo
experiment_name = "whylogs demo"
mlflow.set_experiment(experiment_name)

model_params = {"alpha": 1.0,
                "l1_ratio": 0.7}

lr = ElasticNet(**model_params)
lr.fit(train_x, train_y)
print("ElasticNet model (%s):" % model_params)

ElasticNet model ({'alpha': 1.0, 'l1_ratio': 0.7}):


In [9]:
# run predictions on the batches of data we set up earlier and log whylogs data
for i in range(num_batches):
    with mlflow.start_run(run_name=f"Run {i+1}"):
        batch = subset_test_x[i]
        predicted_output = lr.predict(batch)

        mae = mean_absolute_error(subset_test_y[i], predicted_output)
        print("Subset %.0f, mean absolute error: %s" % (i + 1, mae))

        mlflow.log_params(model_params)
        mlflow.log_metric("mae", mae)

        # use whylogs to log data quality metrics for the current batch
        mlflow.whylogs.log_pandas(batch)

        # wait a second between runs to create a time series of prediction results
        time.sleep(1)

Subset 1, mean absolute error: 1.057122446060594
Subset 2, mean absolute error: 0.5956234001101149
Subset 3, mean absolute error: 0.6190885053112435
Subset 4, mean absolute error: 0.42267661292910663
Subset 5, mean absolute error: 0.4312880712119783
Subset 6, mean absolute error: 0.6578400675841664
Subset 7, mean absolute error: 0.5532824018430903
Subset 8, mean absolute error: 0.5781854068438411
Subset 9, mean absolute error: 0.8391819079712775
Subset 10, mean absolute error: 0.6096163556316625
Subset 11, mean absolute error: 0.6621457967256024
Subset 12, mean absolute error: 1.0111946685519446
Subset 13, mean absolute error: 0.3297452898145677
Subset 14, mean absolute error: 0.559885393804619
Subset 15, mean absolute error: 0.6205237483583886
Subset 16, mean absolute error: 0.7550771198401414
Subset 17, mean absolute error: 0.4607105536784569
Subset 18, mean absolute error: 0.43523432540350876
Subset 19, mean absolute error: 0.8384642864477048
Subset 20, mean absolute error: 0.928165

# Accessing whylogs Data From Your Experiment
Now, let's explore our whylogs data inside the MLflow experiment.

In [ ]:
client = mlflow.tracking.MlflowClient()
experiment = client.get_experiment_by_name(experiment_name)
experiment.name, experiment.experiment_id

MLflow stores the data profiles as *artifacts*. These can be retrieved in the same way you access MLflow projects, parameters, and metrics.

whylogs exposes helper API for accessing whylogs-specific output of an experiment.

In [ ]:
whylogs.mlflow.list_whylogs_runs(experiment.experiment_id)

# Visualizing whylogs Data
Our integration allows you to quickly collect the statistical profiles produced during experimentation.

In [ ]:
mlflow_profiles = whylogs.mlflow.get_experiment_profiles(experiment.experiment_id)
mlflow_profiles

You can then use `whylogs.viz` to easily produce visualizations for the whylogs profile data.

Below, you can see how the data changed over time in our batches for the column called `free sulfur dioxide`.

In [ ]:
from whylogs.viz import ProfileVisualizer

viz = ProfileVisualizer()
viz.set_profiles(mlflow_profiles)

In [ ]:
viz.plot_distribution("free sulfur dioxide", ts_format="%d-%b-%y %H:%M:%S")

In [ ]:
viz.plot_uniqueness("free sulfur dioxide", ts_format="%d-%b-%y %H:%M:%S")

In [ ]:
viz.plot_missing_values("free sulfur dioxide", ts_format="%d-%b-%y %H:%M:%S")

In [ ]:
viz.plot_data_types("free sulfur dioxide", ts_format="%d-%b-%y %H:%M:%S")

We can also plot the mean error of each run for comparison.

In [ ]:
import matplotlib.pyplot as plt

plt.close('all')

runs = mlflow.search_runs(experiment.experiment_id)
plt.figure(figsize=(10,2))
plt.plot(runs['start_time'], runs['metrics.mae'])
plt.show()

With whylogs, collecting and visualizing data quality metrics at both training and inference time for your MLflow runs is made dead simple. These metrics can be invaluable when trying to debug model failures or optimize their performance.

whylogs data can be visualized in more complex ways. Check out [whylogs.viz](https://whylogs.readthedocs.io/en/latest/api/whylogs.viz.html) for details on the API.

In addition, you can also check out how **WhyLabs** can help you visualize data quality metrics by visiting [our sandbox](http://try.whylabsapp.com/). Feel free to reach out to our [slack channel](http://join.slack.whylabs.ai/) if you have any questions!